<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

## Danilo Plazas Irreño

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [1]:
import pandas as pd
import tweepy

In [2]:
# Leer las llaves
keys = pd.read_csv("C:/Users/Danilo/Documents/KonradLorenz/NLP/Clase12/mis_llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'bearer_token', 'access_token', 'access_token_secret'])

In [3]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [30]:
tweets = tweepy.Cursor(api.search,
                       q=['River -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,200km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [31]:
# Objeto generador
t = [tweet for tweet in tweets]

In [32]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    #temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)

In [46]:
t_processed[99]

['@SantaFe Salieron con miedo, River con 6 minutos 2 goles',
 datetime.datetime(2021, 5, 20, 0, 9, 16),
 'Bogotá, DC, Colombia',
 1395169719657828352,
 'https://twitter.com/i/web/stuatus/1395169719657828352']

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [48]:
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'user_location', 'id', 'link']
data.head()

,tweets,date,user_location,id,link
0,@jfaridnageg @SantaFe Uy no que grosería ese e...,2021-05-20 00:12:04,Colombia,1395170422493167616,https://twitter.com/i/web/stuatus/139517042249...
1,"Es mentira, todos sabíamos que river iba ganar...",2021-05-20 00:11:57,"Bogotá, D.C., Colombia",1395170394039062529,https://twitter.com/i/web/stuatus/139517039403...
2,"@Calihea1 Grande Santa Fe, dejándose ganar en ...",2021-05-20 00:11:55,"Bogotá, D.C., Colombia",1395170385843400705,https://twitter.com/i/web/stuatus/139517038584...
3,El amor que tienen los jugadores de river a es...,2021-05-20 00:11:55,"Bogotá, D.C., Colombia",1395170385059008515,https://twitter.com/i/web/stuatus/139517038505...
4,@JFCadavid 😆😆😆😆😆Pensaron que River se iba a me...,2021-05-20 00:11:53,"Cali, Colombia",1395170377626697728,https://twitter.com/i/web/stuatus/139517037762...


In [49]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

In [51]:
data['emojis'] = data['tweets'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,user_location,id,link,emojis
0,@jfaridnageg @SantaFe Uy no que grosería ese e...,2021-05-20 00:12:04,Colombia,1395170422493167616,https://twitter.com/i/web/stuatus/139517042249...,[]
1,"Es mentira, todos sabíamos que river iba ganar...",2021-05-20 00:11:57,"Bogotá, D.C., Colombia",1395170394039062529,https://twitter.com/i/web/stuatus/139517039403...,[]
2,"@Calihea1 Grande Santa Fe, dejándose ganar en ...",2021-05-20 00:11:55,"Bogotá, D.C., Colombia",1395170385843400705,https://twitter.com/i/web/stuatus/139517038584...,[🤡]
3,El amor que tienen los jugadores de river a es...,2021-05-20 00:11:55,"Bogotá, D.C., Colombia",1395170385059008515,https://twitter.com/i/web/stuatus/139517038505...,[]
4,@JFCadavid 😆😆😆😆😆Pensaron que River se iba a me...,2021-05-20 00:11:53,"Cali, Colombia",1395170377626697728,https://twitter.com/i/web/stuatus/139517037762...,"[😆, 😆, 😆, 😆, 😆]"


In [52]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [54]:
from collections import Counter
temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(), Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending = False, inplace = True)

In [57]:
trace = go.Bar(x=temp_emojis.emoji.values, y=temp_emojis.cnt.values)
layout = go.Layout(title = "Emojis más usados en el partido de River vs Santa Fe")
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)